In [13]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [14]:
import os
import sys

sys.path.append('/coc/scratch/anarayanan68/mint/')

In [15]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, initializers, regularizers

from mint.core import fact_model, base_model_util, base_models, primitive_models, model_builder
from mint.utils import inputs_util, config_util

In [16]:
config_path = '/coc/scratch/anarayanan68/mint/configs/audio_based_blending__embed_based_conditioning.config'
enc_config_path = '/coc/scratch/anarayanan68/mint/configs/audio_based_blending__embed_based_conditioning-enc_config.yml'

In [17]:
expt_root = '/coc/scratch/anarayanan68/mint/_expts/audio_based_blending__embed_based_conditioning'

checkpoint_dir = os.path.join(expt_root, 'checkpoints/')

In [18]:
configs = config_util.get_configs_from_pipeline_file(config_path)
model_config = configs['model']
eval_config = configs['eval_config']
eval_dataset_config = configs['eval_dataset']

In [19]:
enc_config_yaml = config_util.read_yaml_config(enc_config_path)

In [20]:
# Model build & restore

model = model_builder.build(model_config, is_training=False,
    encoder_config_yaml=enc_config_yaml, dataset_config=eval_dataset_config)

checkpoint_manager=tf.train.CheckpointManager(
    tf.train.Checkpoint(model=model),
    directory=checkpoint_dir,
    max_to_keep=None)

checkpoint_manager.restore_or_initialize()

'/coc/scratch/anarayanan68/mint/_expts/audio_based_blending__embed_based_conditioning/checkpoints/ckpt-999'

In [21]:
num_primitives = enc_config_yaml['num_primitives']
all_primitives = model.blend_vec_to_seq(np.eye(num_primitives))
all_primitives

<tf.Tensor: shape=(100, 120, 147), dtype=float32, numpy=
array([[[0.0063658 , 0.00708202, 0.00642115, ..., 0.00707373,
         0.008091  , 0.00658407],
        [0.00637796, 0.00713212, 0.00654726, ..., 0.00713249,
         0.00817177, 0.00657123],
        [0.00630593, 0.0070834 , 0.0064329 , ..., 0.00715862,
         0.00820279, 0.00660632],
        ...,
        [0.00632795, 0.00709465, 0.00646312, ..., 0.00714195,
         0.00826836, 0.00659244],
        [0.00644606, 0.00715921, 0.00637297, ..., 0.00712748,
         0.00821426, 0.00658827],
        [0.00628852, 0.00714657, 0.00656557, ..., 0.00713495,
         0.0080614 , 0.00655423]],

       [[0.00752923, 0.00752923, 0.00752923, ..., 0.00752923,
         0.00752923, 0.00752922],
        [0.00752923, 0.00752923, 0.00752923, ..., 0.00752923,
         0.00752923, 0.00752922],
        [0.00752923, 0.00752923, 0.00752923, ..., 0.00752923,
         0.00752923, 0.00752922],
        ...,
        [0.00752923, 0.00752923, 0.00752923, ..., 0

In [12]:
save_dir = os.path.join(expt_root, 'primitives')
os.makedirs(save_dir, exist_ok=True)
for i in range(num_primitives):
    np.save(os.path.join(save_dir, f"PRIMITIVE--{i+1}_of_{num_primitives}.npy"), all_primitives[i])